In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchinfo import summary
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris


from datetime import datetime as dt
import os
import time
import sys
sys.path.append('..')

from utils.functions import *
from models.TryNets import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
train_data   = CustomMawiDataset(year='2016', month='01', as_matrix=True)
test_data    = CustomMawiDataset(year='2016', month='02', as_matrix=True)

torch.Size([514040, 1, 7, 7])
torch.Size([514040])
torch.Size([383050, 1, 7, 7])
torch.Size([383050])


In [22]:
class MawiNet(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Sequential(
                nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0),
                nn.BatchNorm2d(8),
                nn.ReLU()),                    

            nn.Sequential(
                nn.Flatten(),
                nn.Linear(1152, num_classes))
        )
    
    def forward(self, x):
        return self.layers(x) 
    
model = MawiNet().to(device)    
summary(model, (1, 1, 7, 7))
summary(model)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 2, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Flatten: 3]

In [24]:
x = torch.randn(1, 1, 7, 7).to(device)
print(x.shape)

x = model.layers[0](x)
print(x.shape)
# x = model.layers[1](x)
# print(x.shape)
# x = model.layers[2](x)
# print(x.shape)

print(x.shape[-1] * x.shape[-2] * x.shape[-3])

x = torch.randn(1, 1, 7, 7).to(device)
x = model(x)

torch.Size([1, 1, 7, 7])
torch.Size([1, 8, 5, 5])
200


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x200 and 1152x2)

In [ ]:
epochs = 5
batch_size = 300

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
lr=0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

import time
start_time = time.time()

seq = 0
for i in range(epochs):
    trn_cor = 0
    trn_cnt = 0
    tst_cor = 0
    tst_cnt = 0
        
    for b, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        b+=1
            
        y_pred = model(X_train)

        # print(y_pred)
        # int(y_train)

        loss = criterion(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        predicted = torch.max(y_pred.data, 1)[1]
        batch_cor = (predicted == y_train).sum()
        trn_cor += batch_cor
        trn_cnt += len(predicted)

        cnf = torch.mean(torch.max(nn.functional.softmax(y_pred, dim=-1), 1)[0]).item()
            
        if (b-1)%10 == 0:
            print(f'Epoch: {i:2} Batch: {b:3} Loss: {loss.item():4.4f} Accuracy Train: {trn_cor.item()*100/trn_cnt:2.3f}%')
            
        seq += 1

print(f'\nDuration: {time.time() - start_time:.0f} seconds')